## Objectives
- Binger classification을 위한 준비 단계로서 필요한 노트북이다.
- 최초로 로그인 한 후 가장 첫 세션에서 도달한 레벨을 first_session_level로 정의한다.

In [1]:
from config import *

In [2]:
levelups_ref = pd.read_csv("../data/levelups_panel_4.csv", encoding="utf-8")

In [3]:
l_sample = levelups_ref.player_slug.unique()
len(l_sample)

59342

* Login history

In [4]:
# login history: character
logins = pd.read_csv(data_dir+"characters_login.csv", encoding="utf-8")
logins.drop(columns=["id","player_id","character_id","event_timestamp"], inplace=True)
logins = simple_preprocess(logins)

logins = logins.query("player_slug in @l_sample").reset_index(drop=True)

In [5]:
# register date
tmp = levelups_ref.query("level_from==1")[["player_slug","character_slug","event_datetime_prev"]]
tmp = tmp.sort_values(by=["event_datetime_prev"]).reset_index(drop=True)
tmp.rename(columns={"event_datetime_prev":"register_datetime"}, inplace=True)
tmp["register_datetime"] = pd.to_datetime(tmp["register_datetime"])
tmp

,player_slug,character_slug,register_datetime
0,2021-11-01T07:31:49.366Z,레드스완,2021-11-01 16:33:19
1,2021-11-01T07:33:20.616Z,칸토,2021-11-01 16:42:35
2,2021-11-01T08:06:03.864Z,무지개,2021-11-01 17:07:35
3,2021-11-01T08:32:01.518Z,성주,2021-11-01 17:32:37
4,2021-11-01T08:49:02.090Z,뚫뚧,2021-11-01 17:51:36
...,...,...,...
59337,2022-03-30T14:09:03.502Z,별빛처럼,2022-03-30 23:11:13
59338,2022-03-30T14:13:23.546Z,sheal,2022-03-30 23:14:32
59339,2022-03-30T14:20:04.049Z,타이풍,2022-03-30 23:21:46
59340,2022-01-24T10:59:49.532Z,거인,2022-03-30 23:22:59


In [6]:
# next login date

def match_next_login(row):
    logins_ = logins.query("player_slug==@row.player_slug & character_slug==@row.character_slug & event_datetime>@row.register_datetime")
    logins_ = logins_["event_datetime"].tolist()
    
    # if there is no next login
    if len(logins_)==0:
        return None
    
    # if there is next login(s)
    else:
        for login_ in logins_:
            dur = (login_ - row.register_datetime).total_seconds() / 3600
            if dur > 1: # 1시간 이내 재로그인은 무시
                break
            
        if dur <= 1:
            return None
        else:
            return login_

tmp["next_login_datetime"] = tmp.progress_apply(match_next_login, axis=1)

100%|████████████████████████████████████████████████████████████████████████████| 59342/59342 [11:48<00:00, 83.76it/s]


In [7]:
tmp = tmp.sort_values(by="next_login_datetime").reset_index(drop=True)
tmp

,player_slug,character_slug,register_datetime,next_login_datetime
0,2021-11-01T10:19:54.128Z,박병호,2021-11-01 19:22:47,2021-11-01 21:34:29
1,2021-11-01T11:36:26.389Z,불탄법사,2021-11-01 20:38:40,2021-11-01 21:56:36
2,2021-11-01T08:49:02.090Z,뚫뚧,2021-11-01 17:51:36,2021-11-01 23:37:31
3,2021-11-01T14:12:34.729Z,레서판다,2021-11-01 23:15:11,2021-11-02 01:45:22
4,2021-11-01T19:28:04.508Z,옹심이,2021-11-02 04:28:53,2021-11-02 05:38:36
...,...,...,...,...
59337,2022-03-30T14:09:03.502Z,별빛처럼,2022-03-30 23:11:13,NaT
59338,2022-03-30T14:13:23.546Z,sheal,2022-03-30 23:14:32,NaT
59339,2022-03-30T14:20:04.049Z,타이풍,2022-03-30 23:21:46,NaT
59340,2022-01-24T10:59:49.532Z,거인,2022-03-30 23:22:59,NaT


* Next login 있었던 경우

In [8]:
# last level up before the next login date
tmp_ = levelups_ref[["player_slug","character_slug","event_datetime","level_to"]].copy()
tmp_["event_datetime"] = pd.to_datetime(tmp_["event_datetime"])
tmp_ = tmp_.sort_values(by=["event_datetime"]).reset_index(drop=True)
tmp_.rename(columns={"event_datetime":"next_login_datetime"}, inplace=True)

tmp1 = pd.merge_asof(tmp.query("next_login_datetime==next_login_datetime"), tmp_,
                     on="next_login_datetime", by=["player_slug","character_slug"], direction="backward")
tmp1["level_to"].fillna(1, inplace=True)
tmp1

,player_slug,character_slug,register_datetime,next_login_datetime,level_to
0,2021-11-01T10:19:54.128Z,박병호,2021-11-01 19:22:47,2021-11-01 21:34:29,42.0
1,2021-11-01T11:36:26.389Z,불탄법사,2021-11-01 20:38:40,2021-11-01 21:56:36,17.0
2,2021-11-01T08:49:02.090Z,뚫뚧,2021-11-01 17:51:36,2021-11-01 23:37:31,6.0
3,2021-11-01T14:12:34.729Z,레서판다,2021-11-01 23:15:11,2021-11-02 01:45:22,71.0
4,2021-11-01T19:28:04.508Z,옹심이,2021-11-02 04:28:53,2021-11-02 05:38:36,48.0
...,...,...,...,...,...
27508,2022-03-17T23:31:51.223Z,마스터,2022-03-18 08:35:22,2022-03-30 20:09:56,61.0
27509,2021-12-19T14:51:01.714Z,Timidy,2022-03-29 15:20:22,2022-03-30 20:42:52,58.0
27510,2022-03-22T14:57:57.373Z,산라쿠,2022-03-22 23:58:45,2022-03-30 22:15:26,5.0
27511,2022-03-30T11:15:48.145Z,어리버리,2022-03-30 20:22:27,2022-03-30 23:23:53,93.0


* Next login 없었던 경우

In [9]:
tmp2 = tmp.query("next_login_datetime!=next_login_datetime")
tmp2 = pd.merge(tmp2,
                levelups_ref[["player_slug","character_slug","highest_level"]].rename(columns={"highest_level":"level_to"}).drop_duplicates(),
                how="left")
tmp2

,player_slug,character_slug,register_datetime,next_login_datetime,level_to
0,2021-11-01T07:31:49.366Z,레드스완,2021-11-01 16:33:19,NaT,51
1,2021-11-01T07:33:20.616Z,칸토,2021-11-01 16:42:35,NaT,21
2,2021-11-01T08:06:03.864Z,무지개,2021-11-01 17:07:35,NaT,7
3,2021-11-01T08:32:01.518Z,성주,2021-11-01 17:32:37,NaT,100
4,2021-11-01T09:26:02.920Z,킹바나나,2021-11-01 18:26:49,NaT,10
...,...,...,...,...,...
31824,2022-03-30T14:09:03.502Z,별빛처럼,2022-03-30 23:11:13,NaT,12
31825,2022-03-30T14:13:23.546Z,sheal,2022-03-30 23:14:32,NaT,22
31826,2022-03-30T14:20:04.049Z,타이풍,2022-03-30 23:21:46,NaT,25
31827,2022-01-24T10:59:49.532Z,거인,2022-03-30 23:22:59,NaT,11


In [10]:
tmp = pd.concat([tmp1, tmp2]).sort_index()
tmp.rename(columns={"level_to":"first_session_level"}, inplace=True)
tmp["first_session_level"] = tmp["first_session_level"].astype(int)
tmp

,player_slug,character_slug,register_datetime,next_login_datetime,first_session_level
0,2021-11-01T10:19:54.128Z,박병호,2021-11-01 19:22:47,2021-11-01 21:34:29,42
0,2021-11-01T07:31:49.366Z,레드스완,2021-11-01 16:33:19,NaT,51
1,2021-11-01T07:33:20.616Z,칸토,2021-11-01 16:42:35,NaT,21
1,2021-11-01T11:36:26.389Z,불탄법사,2021-11-01 20:38:40,2021-11-01 21:56:36,17
2,2021-11-01T08:49:02.090Z,뚫뚧,2021-11-01 17:51:36,2021-11-01 23:37:31,6
...,...,...,...,...,...
31824,2022-03-30T14:09:03.502Z,별빛처럼,2022-03-30 23:11:13,NaT,12
31825,2022-03-30T14:13:23.546Z,sheal,2022-03-30 23:14:32,NaT,22
31826,2022-03-30T14:20:04.049Z,타이풍,2022-03-30 23:21:46,NaT,25
31827,2022-01-24T10:59:49.532Z,거인,2022-03-30 23:22:59,NaT,11


In [11]:
levelups_ref_5 = pd.merge(tmp[["player_slug","character_slug","first_session_level"]], levelups_ref, how="right")
levelups_ref_5

,player_slug,character_slug,first_session_level,total_purchase,total_ad,highest_level,status,level_from,level_to,event_datetime,death,ad_revival,ad_item,ad_total,purchase,event_datetime_prev,interval,interval_cum,playtime_boss,playtime_dungeon,playtime_tower,playtime_pvp,playtime_farm,playtime_survival,playtime_side
0,2021-10-31T16:01:57.631Z,킹덤,2,0.0,0.0,7,complete,1,2,2021-11-04 23:42:28,0.0,0.0,0.0,0.0,0.0,2021-11-04 23:41:55,0.009444,0.009444,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-10-31T16:01:57.631Z,킹덤,2,0.0,0.0,7,complete,2,3,2021-11-13 07:50:45,0.0,0.0,0.0,0.0,0.0,2021-11-04 23:42:28,200.138333,200.147778,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-10-31T16:01:57.631Z,킹덤,2,0.0,0.0,7,complete,3,4,2021-11-13 07:52:19,0.0,0.0,0.0,0.0,0.0,2021-11-13 07:50:45,0.026389,200.174167,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-10-31T16:01:57.631Z,킹덤,2,0.0,0.0,7,complete,4,5,2021-11-13 13:11:55,0.0,0.0,0.0,0.0,0.0,2021-11-13 07:52:19,5.326944,205.501111,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-10-31T16:01:57.631Z,킹덤,2,0.0,0.0,7,complete,5,6,2021-11-13 13:15:02,0.0,0.0,0.0,0.0,0.0,2021-11-13 13:11:55,0.052222,205.553333,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466992,2022-03-30T14:36:22.522Z,킹키킹,8,0.0,0.0,8,complete,3,4,2022-03-30 23:44:15,0.0,0.0,0.0,0.0,0.0,2022-03-30 23:42:14,0.033889,0.084444,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2466993,2022-03-30T14:36:22.522Z,킹키킹,8,0.0,0.0,8,complete,4,5,2022-03-30 23:46:19,0.0,0.0,0.0,0.0,0.0,2022-03-30 23:44:15,0.034722,0.119167,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2466994,2022-03-30T14:36:22.522Z,킹키킹,8,0.0,0.0,8,complete,5,6,2022-03-30 23:47:29,0.0,0.0,0.0,0.0,0.0,2022-03-30 23:46:19,0.019722,0.138889,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2466995,2022-03-30T14:36:22.522Z,킹키킹,8,0.0,0.0,8,complete,6,7,2022-03-30 23:48:48,0.0,0.0,0.0,0.0,0.0,2022-03-30 23:47:29,0.022222,0.161111,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
levelups_ref_5.to_csv("../data/levelups_panel_5.csv", index=None, encoding="utf-8")